In [289]:
import straw
import pandas as pd 
import numpy as np

In [290]:
filename = "CD69negDPWTR2"#"CD69posCD4SPWTR1" #CD69negDPWTR1
path_save="/Users/ealjibur/OneDrive - Imperial College London/HiC/Hic_ya/"
path_to_hic=path_save+filename+".hic"
path_to_ABwig="~/downloads/"+filename+"_eigenvector_100kb.wig"

#path_to_ABwig is path to eigen vector, we expect that the eigenvector 
#is positive in A comp ad negative in B so it may need to be reorientated

In [291]:
#parameters 
Range, resolution =100000, 5000
AB_res = 100000

In [292]:
#initialising Hi-C file ad chromosome dictionary.
HiC_Matrix = straw.straw(path_to_hic)
chrDict = HiC_Matrix.chromDotSizes.data
chrDict.pop("All")
chrDict.pop("chrM")

#loading A/B file and initialising an index for merging 
AB_file = pd.read_csv(path_to_ABwig, skiprows=1,sep='\t',names=["chr","start","end","score"])
AB_file["x"]= (AB_file.start-1)/5000 #

HiC version:  8


In [293]:
All_chroms = pd.DataFrame({"Chromosome":pd.Series([], dtype='str'),
                           "Start":pd.Series([], dtype='int'),
                           "End":pd.Series([], dtype='int'), 
                           "values":pd.Series([], dtype='float')})
contacts = 0 

for chromosome in chrDict.keys():
    #get chromosome information from HiC
    norm_hic= HiC_Matrix.getNormalizedMatrix(chromosome,chromosome, "KR", "BP",resolution)
    xpos, ypos, vals = norm_hic.getDataFromGenomeRegion(0,chrDict[chromosome][1], 0,chrDict[chromosome][1])
    curr = pd.DataFrame({"x":xpos, "y": ypos, "values": vals}) 
    contacts+= curr["values"].sum()

    #filter outside of local range 
    curr= curr[resolution*np.abs(curr.x-curr.y)<=Range] 

    #mirror x and y to make fully symetric matrix 
    curr1 =curr.copy()
    curr1.rename(columns={"x": "y", "y": "x"}, inplace=True)
    curr=pd.concat([curr1,curr])
    curr.drop(columns=["y"], inplace=True)

    #sum together local contacts 
    curr=curr.groupby("x").sum()
    curr.reset_index(inplace=True)

    #get normalisation from AB
    curr["floorx"]=int(AB_res/resolution)*np.floor(curr["x"]/int(AB_res/resolution)) #this is just for merging with A/B wig 
    joinedwithAB = curr.merge(AB_file[AB_file.chr==chromosome], right_on="x", left_on="floorx",  how="left")
    chromosome_norm = joinedwithAB[np.sign(joinedwithAB.score)==-1]["values"].mean()

    #normalise
    curr["values"] = curr["values"]-chromosome_norm
    curr["values"] = curr["values"].rolling(3).mean()

    #clean matrix
    curr["x"] = resolution*curr["x"]+1
    curr["Chromosome"]=chromosome
    curr.rename(columns={"x": "Start"}, inplace=True)
    curr.drop(columns=["floorx"],inplace=True)
    curr["End"]=curr.Start+resolution-1
    
    #add to pandasas array containing full list 
    All_chroms = All_chroms.append(curr, ignore_index=True)

In [294]:
#save 
All_chroms["values"] = All_chroms["values"]*1e6/contacts
All_chroms[["Chromosome","Start","End", "values"]].to_csv(filename+".wig", sep='\t',header=False, index=False)